# Protein Folding using Genetic Algorithm (GA) on HP Model

This notebook processes protein sequences in **FASTA format** and applies a **Genetic Algorithm** to find low-energy foldings under the HP lattice model.  
The program supports:

- **Square 2D Lattice**
- **Triangular 2D Lattice**
- **Cubic 3D Lattice**

The steps include:
1. Loading `.fasta` sequence files.
2. Converting amino acid sequences to HP sequences.
3. Running the GA-based folding method for each lattice type.
4. Calculating minimum energies and deviations from benchmark optimal energies.
5. Visualizing the resulting protein folds.

## **Dataset**

The dataset contains protein sequences in **FASTA format** from the RCSB Protein Data Bank.

| File Name             | Protein ID | Chain | Protein Name | Organism              | Sequence Length | Sequence                                                                 |
|-----------------------|-----------|-------|--------------|-----------------------|----------------|---------------------------------------------------------------------------|
| rcsb_pdb_1A7F.fasta   | 1A7F_1    | A     | Insulin      | Homo sapiens (9606)   | 21             | GIVEQCCTSICSLYQLENYCN                                                    |
| rcsb_pdb_1A7F.fasta   | 1A7F_2    | B     | Insulin      | Homo sapiens (9606)   | 30             | FVNQHLCGSHLVEALELVCGERGGFYTPK                                            |
| rcsb_pdb_1APH.fasta   | 1APH_1    | A     | Insulin      | Bos taurus (9913)                | 21              | GIVEQCCASVCSLYQLENYCN   
| rcsb_pdb_1APH.fasta   | 1APH_2    | B     | Insulin      | Bos taurus (9913)                | 30              | FVNQHLCGSHLVEALYLVCGERGFFYTPKA
| rcsb_pdb_1B17.fasta   | 1B17_1    | A     | Insulin      | Sus scrofa (9823)                | 21              | GIVEQCCTSICSLYQLENYCN
| rcsb_pdb_1B17.fasta   | 1B17_2    | B     | Insulin      | Sus scrofa (9823)                | 30              | FVNQHLCGSHLVEALYLVCGERGFFYTPKA


**Notes:**
- **Protein ID**: Taken from the FASTA header.
- **Chain**: The specific chain label in the protein structure.
- **Sequence Length**: Number of amino acids in the chain.
- **Sequence**: Single-letter amino acid codes.

These sequences are converted to **HP sequences** where:
- **H** = Hydrophobic residues: {A, G, I, L, M, F, P, W, V}
- **P** = Polar residues: {R, N, D, C, E, Q, H, K, S, T, Y}

## **Methodology**

### HP Model Representation
Proteins are simplified to a binary sequence:
- **H** (hydrophobic)
- **P** (polar)

### Lattice Types
1. **Square 2D Lattice** : movement allowed in 4 directions: up, down, left, right.
2. **Triangular 2D Lattice** : movement allowed in 6 directions at 60° intervals.
3. **Cubic 3D Lattice** : movement allowed in 6 directions in 3D space.

### Genetic Algorithm (GA) Workflow
1. **Initialization** : generate a random population of folds.
2. **Fitness Evaluation** : calculate HP model energy + compactness penalty.
3. **Selection** : tournament selection to choose parents.
4. **Crossover** : multi-point crossover to create offspring.
5. **Mutation** : introduce random changes to maintain diversity.
6. **Replacement** : keep best folds and remove duplicates.
7. **Termination** : after a fixed number of generations.

### Energy Calculation
Energy is based on the number of **non-consecutive H–H contacts**.  
- More H–H contacts → lower energy (more stable fold).

### Visualization
- **2D plots** for square & triangular lattices.
- **3D plots** for cubic lattices.


The code takes sequences in **FASTA** or **HP** format, folds them using GA, calculates energies, and visualizes the best fold.


## 1- Importing Libraries

We use:
- `random` & `heapq` for GA operations
- `matplotlib` for visualization
- `math` for lattice geometry
- `time` for runtime measurement
- `sys` for program control
- `mpl_toolkits.mplot3d` for 3D visualization


In [ ]:
import random
import heapq
import matplotlib.pyplot as plt
import math
import sys
import time
from mpl_toolkits.mplot3d import Axes3D

## 2-Lattice Directions

Defines movement directions for:
- **Square 2D lattice** — 4 directions (up, right, down, left)
- **Cubic 3D lattice** — 6 directions in 3D
- **Triangular 2D lattice** — 6 directions at 60° intervals


In [ ]:
# Directions for 2D lattice
square_directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

## 3D Cubic Directions
cubic_directions = [(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)]

# Directions for 2D Trianguler Lattice:
trianguler_directions = [
    (1.0, 0.0),                                     # 0: Right
    (0.5, math.sqrt(3)/2),                          # 1: Up-Right
    (-0.5, math.sqrt(3)/2),                         # 2: Up-Left
    (-1.0, 0.0),                                    # 3: Left
    (-0.5, -math.sqrt(3)/2),                        # 4: Down-Left
    (0.5, -math.sqrt(3)/2)                          # 5: Down-Right
]

## 3- Benchmark Energies

Known optimal energies for specific sequence lengths in 2D and 3D lattices.  
Used to calculate deviation from optimal folding results.


In [ ]:
optimal_2d = {
    20: -9, 
    24: -9, 
    25: -8, 
    36: -14,
    48: -23, 
    50: -21, 
    60: -36,
    64: -42, 
    85: -53
}

optimal_3d = {
    20: -11, 
    24: -13, 
    25: -9, 
    36: -18, 
    48: -31, 
    50: -34, 
    60: -55, 
    64: -59
}

## 4- Converting FASTA Sequence to HP Sequence

Maps amino acids to:
- **H** (Hydrophobic): A, G, I, L, M, F, P, W, V
- **P** (Polar): R, N, D, C, E, Q, H, K, S, T, Y


In [ ]:
def fasta_to_hp(fasta_sequence):
    """
    Converts a FASTA amino acid sequence to HP model using:
    H = Hydrophobic {A, G, I, L, M, F, P, W, V}
    P = Polar       {R, N, D, C, E, Q, H, K, S, T, Y}
    """
    
    h_residues = {'A', 'G','I', 'L', 'M', 'F', 'P', 'W', 'V'}
    p_residues = {'R', 'N', 'D', 'C', 'E', 'Q', 'H','K', 'S', 'T', 'Y'}
    
    sequence = fasta_sequence.upper()
    
    hp_sequence = ''
    
    # aa : amino acid
    for aa in sequence:
        if aa in h_residues:
            hp_sequence += 'H'
        elif aa in p_residues:
            hp_sequence += 'P'
        else:
            raise ValueError(f"Unkown amino acid '{aa}' in sequence. ")
    return hp_sequence

## 5- Folding Functions

Functions for generating protein coordinates from HP sequences:
- **Square 2D folding**


In [ ]:
def square_fold_protein(sequence, moves):
    square_direction = 0
    square_position = (0, 0)
    square_coords = [square_position]
    square_occupied = {square_position}
    for move in moves:
        square_direction = (square_direction - 1) % 4 if move == 1 else (square_direction + 1) % 4 if move == 2 else square_direction
        dx, dy = square_directions[square_direction]
        square_position = (square_position[0] + dx, square_position[1] + dy)
        if square_position in square_occupied:
            return None
        square_coords.append(square_position)
        square_occupied.add(square_position)
    return square_coord

- **Triangular 2D folding**

In [ ]:
def rotate_yaw_left(d): x, y, z = d; return (-z, y, x)
def rotate_yaw_right(d): x, y, z = d; return (z, y, -x)
def rotate_pitch_up(d): x, y, z = d; return (x, -z, y)
def rotate_pitch_down(d): x, y, z = d; return (x, z, -y)
def reverse_direction(d): return tuple(-v for v in d)

def fold_protein_3d_full(seq, moves):
    pos, coords, occupied = (0, 0, 0), [(0, 0, 0)], {(0, 0, 0)}
    direction = (1, 0, 0)
    for move in moves:
        if move == 1: direction = rotate_yaw_left(direction)
        elif move == 2: direction = rotate_yaw_right(direction)
        elif move == 3: direction = rotate_pitch_up(direction)
        elif move == 4: direction = rotate_pitch_down(direction)
        elif move == 5: direction = reverse_direction(direction)
        dx, dy, dz = direction
        pos = (pos[0] + dx, pos[1] + dy, pos[2] + dz)
        if pos in occupied: return None
        coords.append(pos)
        occupied.add(pos)
    return coords

- **Cubic 3D folding** (with rotations)

In [ ]:
def trianguler_fold_protein(sequence, moves):
    trianguler_direction = 0  # Start facing right (index 0)
    trianguler_position = (0.0, 0.0)
    trianguler_coords = [trianguler_position]
    trianguler_occupied = {trianguler_position}

    for move in moves:
        if move == 1:
            trianguler_direction = (trianguler_direction + 1) % 6  # Left turn (60°)
        elif move == 2:
            trianguler_direction = (trianguler_direction - 1) % 6  # Right turn (-60°)

        dx, dy = trianguler_directions[trianguler_direction]
        new_position = (round(trianguler_position[0] + dx, 5), round(trianguler_position[1] + dy, 5))

        if new_position in trianguler_occupied:
            return None

        trianguler_coords.append(new_position)
        trianguler_occupied.add(new_position)
        trianguler_position = new_position
        
    return trianguler_coords

## 6- Energy Calculation

Energy is calculated based on non-consecutive H–H contacts:
- More H–H contacts → lower energy.
Includes compactness penalties to prefer compact folds.
